In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [2]:
class LinearClassifier(nn.Module):
    def __init__(self, learning_rate, num_classes, input_dimensions):
        super(LinearClassifier, self).__init__()
        
        self.fully_connected_1 = nn.Linear(*input_dimensions, 128)
        self.fully_connected_2 = nn.Linear(*128, 256)
        self.fully_connected_3 = nn.Linear(*256, num_classes)
        
        self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        self.loss = nn.CrossEntropyLoss()
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, data):
        layer1_output = F.sigmoid(self.fully_connected_1(data))
        layer2_output = F.sigmoid(self.fully_connected_2(layer1_output))
        layer3_output = self.fully_connected_3(layer2_output)
        
        return layer3_output
    
    def learn(self, data, labels):
        self.optimizer.zero_grad()
        data = T.tensor(data).to(self.device)
        labels = T.tensor(labels).to(self.device)
        
        predictions = self.forward(data)
        
        cost = self.loss(predictions, labels)
        
        cost.backward()
        self.optimizer.step()
        

In [3]:
lc = LinearClassifier(0.001, 4, 4)

TypeError: Value after * must be an iterable, not int

In [ ]:
class QAgent():
    def __init__(self, learning_rate = 0.0001, discount_factor = 0.99, explore_exploit_start = 1.0, explore_exploit_min = 0.01, decrement = 1e-5, num_actions = 2, num_states = (4, 1)):
        self.q_classifier = NaiveCartPoleLinearClassifier(learning_rate, num_actions, num_states)
        
        self.learning_rate = learning_rate # alpha
        self.discount_factor = discount_factor # gamma
        self.explore_exploit = explore_exploit_start # epsilon
        print(self.explore_exploit)
        self.num_actions = num_actions
        self.explore_exploit_min = explore_exploit_min
        self.ee_decrement = decrement
        
        self.action_space = [i for i in range(self.num_actions)]
        
        
    def choose_action(self, current_state):
        random_number = np.random.random()
        if random_number < self.explore_exploit:
            return np.random.choice(self.action_space)
        else:
            return T.argmax(self.q_classifier.forward(T.tensor(current_state, dtype=T.float).to(self.q_classifier.device))).item()
            
    def decrement_explore_exploit(self):
        prev_value = self.explore_exploit
        new_value = (self.explore_exploit - self.ee_decrement) if self.explore_exploit > self.explore_exploit_min else self.explore_exploit_min
        self.explore_exploit = new_value
        #print(f"decrement_explore_exploit prev_value: {prev_value} new_value: {new_value} explore_exploit: {self.explore_exploit} ")
        
    def learn(self, current_state, action, reward, new_state):
        #self.q_table[current_state][action] +=  (reward + self.discount_factor * np.max(self.q_classifier(new_state)) - self.q_classifier(current_state)[action])
        self.q_classifier.optimizer.zero_grad()
        current_states = T.tensor(current_state, dtype = T.float).to(self.q_classifier.device)
        actions = T.tensor(action).to(self.q_classifier.device)
        rewards = T.tensor(reward).to(self.q_classifier.device)
        new_states = T.tensor(new_state, dtype = T.float).to(self.q_classifier.device)
        
        predictions = self.q_classifier.forward(current_states)[actions]
        
        next_predictions = self.q_classifier.forward(new_states).max()
        
        next_target = reward + self.discount_factor * next_predictions 
        
        cost = self.q_classifier.loss(next_target, predictions).to(self.q_classifier.device)
        
        cost.backward()
        
        self.q_classifier.optimizer.step()
        
        self.decrement_explore_exploit()
        